<a href="https://colab.research.google.com/github/mtzig/badwriting_OCR/blob/main/iam_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
iam_path = '../drive/MyDrive/semester2/cs766/iam_dataset'

In [4]:
# %cd drive/MyDrive/semester2/cs766/iam_dataset

In [5]:
# %mkdir sentences
# !tar -xf sentences.tar -C sentences

In [6]:
# %mkdir lines
# !tar -xf lines.tar -C lines

In [7]:
# %mkdir ascii
# !tar -xf ascii.tar -C ascii

In [8]:
!git clone https://github.com/mtzig/badwriting_OCR.git
%cd /content/badwriting_OCR/

Cloning into 'badwriting_OCR'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 302 (delta 36), reused 91 (delta 28), pack-reused 192
Receiving objects: 100% (302/302), 8.21 MiB | 21.51 MiB/s, done.
Resolving deltas: 100% (81/81), done.
/content/badwriting_OCR


In [9]:
!pip install -q transformers
!pip install -q datasets jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.4 MB/s eta 0:00:00


In [10]:
from utils import model_utils
from transformers import AdamW
from tqdm.notebook import tqdm
import torch
from PIL import Image

/content/badwriting_OCR/utils/model_utils.py:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for cer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/cer/cer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Code to generate csv file with iam file names and paths

In [12]:
# make a csv file with the file names and text starting from directory sentences
# Filename                       Text
# a01/a01-000u/a01-000u-00.png   A MOVE to stop...

# import csv

# data = [['file_name', 'text']]

# with open(iam_path + '/ascii/lines.txt') as file:
#   for line in file:
#     if line[0] != '#': # ignore instructions
#       file_name = line.split()[0]

#       dash1 = file_name.find('-')
#       dash2 = file_name.find('-', dash1+1)

#       file_name = file_name[:dash1]+'/'+file_name[:dash2]+'/'+file_name+'.png'

#       text = line.split()[-1]
#       text = ' '.join(text.split('|'))
#       data.append([file_name, text])

# csv_file = iam_path + '/iam_data.csv'

# with open(csv_file, mode='w', newline='') as file:
#   writer = csv.writer(file)
#   writer.writerows(data)

In [13]:
train_dataloader,eval_dataloader = model_utils.get_dataloaders(dataset_type='iam', root=iam_path)
model = model_utils.getModel(device)

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
from transformers import TrOCRProcessor
import pandas as pd

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-handwritten")

def compute_outputs(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)


    return pred_str, label_str

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [15]:
base_preds, base_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    base_preds.extend(p)
    base_labels.extend(l)

base_df = pd.DataFrame(data={'preds':base_preds, 'lables':base_labels})
base_df.to_csv('base.csv', index=False)




  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [16]:
full_train_cer = []

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   # train
   model.train()
   train_loss = 0.0
   for batch in tqdm(train_dataloader):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()

   print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

   # evaluate
   model.eval()
   valid_cer = 0.0
   with torch.no_grad():
     for batch in tqdm(eval_dataloader):
       # run batch generation
       outputs = model.generate(batch["pixel_values"].to(device))
       # compute metrics
       cer = model_utils.compute_cer(pred_ids=outputs, label_ids=batch["labels"])
       valid_cer += cer

   full_train_cer.append(cer)

   print("Validation CER:", valid_cer / len(eval_dataloader))

model.save_pretrained(".")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12 [00:00<?, ?it/s]

Loss after epoch 0: 8.185902992884317


  0%|          | 0/4 [00:00<?, ?it/s]

Validation CER: 0.9141996637492871


...

  0%|          | 0/12 [00:00<?, ?it/s]

Loss after epoch 9: 2.0615627268950143


  0%|          | 0/4 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


Validation CER: 1.0378683236732695


In [17]:
full_preds, full_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    full_preds.extend(p)
    full_labels.extend(l)

full_df = pd.DataFrame(data={'preds':full_preds, 'lables':full_labels})
full_df.to_csv('full.csv', index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
model = model_utils.getModel(device)
# for pn, p in model.named_parameters():
#   if pn != 'decoder.output_projection.weight':
#     p.requires_grad = False

for pn, p in model.named_parameters():
  if pn.startswith('encoder'):
    p.requires_grad = False

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
freeze_train_cer = []

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   # train
   model.train()
   train_loss = 0.0
   for batch in tqdm(train_dataloader):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()

   print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

   # evaluate
   model.eval()
   valid_cer = 0.0
   with torch.no_grad():
     for batch in tqdm(eval_dataloader):
       # run batch generation
       outputs = model.generate(batch["pixel_values"].to(device))
       # compute metrics
       cer = model_utils.compute_cer(pred_ids=outputs, label_ids=batch["labels"])
       valid_cer += cer

   freeze_train_cer.append(cer)

   print("Validation CER:", valid_cer / len(eval_dataloader))

model.save_pretrained(".")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12 [00:00<?, ?it/s]

Loss after epoch 0: 8.630637327829996


  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


...

  0%|          | 0/12 [00:00<?, ?it/s]

Loss after epoch 9: 0.529463400443395


  0%|          | 0/4 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


Validation CER: 0.45855693329489594


In [20]:
freeze_preds, freeze_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    freeze_preds.extend(p)
    freeze_labels.extend(l)

freeze_df = pd.DataFrame(data={'preds':freeze_preds, 'lables':freeze_labels})
freeze_df.to_csv('freeze.csv', index=False)

  0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
cer_df = pd.DataFrame(data={'full':full_train_cer, 'freeze':freeze_train_cer})
cer_df.to_csv('cer.csv', index=False)